<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/Scrap_loading_page_with_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.QpVpDVQZ10/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.5BTmHnvtCb/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Ts3RXp2sbk/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as etc
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)
# url = ["https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-0/", "https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-1/", "https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-0/"]
# driver.get(url)

#Get All Links And category

In [3]:
# get all urls for loop
#Fetch the content from the front page
url_base = 'https://www.next.co.uk/children/the-character-shop'
content = requests.get(url_base).content
soup = BeautifulSoup(content, 'html.parser')

In [4]:
upper_data_links = []
down_data_links = []

up_links = soup.find_all('a', class_='hp-btn-invert')
d_links = soup.find_all('a', class_='baby-circular-swipe-img-con')
for link in up_links:
  full_link = urljoin(url_base, link['href'])
  category = link.text
  upper_data_links.append({'category': category, "links": full_link})
for link in d_links:
  full_link = urljoin(url_base, link['href'])
  category = link.text.strip()
  down_data_links.append({'category': category, "links": full_link})

upper_data_links.pop() # remove last one as named "Home" not category
all_links_category = upper_data_links + down_data_links

#Scrap the urls

In [5]:
# # get individual page
def get_link_details(url):
  driver.get(url)
  try:
    composition = driver.find_element(By.ID, 'Composition')
    composition = composition.get_attribute("innerHTML")
  except:
    composition = ""
  try:
    description_head = driver.find_element(By.ID, 'ToneOfVoice')

      # Find all <li> elements inside the 'ToneOfVoice' element
    description_li = description_head.find_elements(By.TAG_NAME, 'li')

    # Iterate through <li> elements and print their text
    full_description = ""
    for element in description_li:
        full_description += element.get_attribute("innerHTML")
  except:
    full_description = ""
  try:
    description_feature = driver.find_element(By.ID, 'ToneOfVoice')
    description_feature = description_feature.get_attribute('outerHTML')
    soup = BeautifulSoup(description_feature, 'html.parser')
    description_feature = soup.get_text()
    description_feature = description_feature.strip()

  except:
    description_feature = ""
  try:
    title_head = driver.find_element(By.CLASS_NAME, 'Title')
    title_head = title_head.find_element(By.TAG_NAME, 'h1')
    title_head = title_head.get_attribute("outerHTML")
    soup = BeautifulSoup(title_head, 'html.parser')
    title_head = soup.get_text()
    title_head = title_head.strip()
  except:
    title_head = ""

  try:
    mydiv = driver.find_element(By.CLASS_NAME, 'shotNavNext')
    next_sibling = mydiv.find_element(By.XPATH, "following-sibling::*[1]")
    image = next_sibling.get_attribute('src')
  except:
    image = ""
  final_description = str(full_description) + " " + str(composition) + " " + str(description_feature)
  return image, title_head,  final_description


In [10]:
def get_data_scrapped(url):
  driver.get(url['links'])
  items = []
  final_data = []

  last_height = driver.execute_script("return document.body.scrollHeight")
  itemTargetCount = 100

  while itemTargetCount > len(items):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
      break
    last_height = new_height
    elements = driver.find_elements(By.CLASS_NAME, 'produc-1mup83m')
    # Create a list to store the src attributes
    href_links = []
    for index, element in enumerate(elements):
        # image url from the pages list
          try:
            href = element.get_attribute("href")
            href_links.append(href)
            # image, title_head, final_description  = get_link_details(href)
            # final_data.append({"title": title_head, "category": url['category'], "description": final_description, "image": image})
          except:
            pass
    # print(len(href_links))
  return href_links
final_links_scraped = []
for index, link in enumerate(all_links_category):
  if index < 2:
    links_array = get_data_scrapped(link)
    final_links_scraped += links_array
    # data_scrapped = get_data_scrapped(link)
    # final_all_scrapped.append(data_scrapped)

print(len(final_links_scraped))


1321


In [11]:
for links in final_links_scraped:
  print(links)

https://www.next.co.uk/style/ls298607/p99506#p99506
https://www.next.co.uk/style/st548099/244771#244771
https://www.next.co.uk/style/st548099/d35903#d35903
https://www.next.co.uk/style/st636563/t24910#t24910
https://www.next.co.uk/g224216s4/d99916#d99916
https://www.next.co.uk/g624218s7/d99928#d99928
https://www.next.co.uk/g624211s11/d99925#d99925
https://www.next.co.uk/style/st548096/852315#852315
https://www.next.co.uk/style/st548096/d35916#d35916
https://www.next.co.uk/g62469s1/d73942#d73942
https://www.next.co.uk/g224213s7/d99918#d99918
https://www.next.co.uk/style/st555398/244057#244057
https://www.next.co.uk/style/st555398/a88819#a88819
https://www.next.co.uk/g624208s2/d99920#d99920
https://www.next.co.uk/g624218s9/d99919#d99919
https://www.next.co.uk/g624223s3/d99804#d99804
https://www.next.co.uk/style/st160374/u67051#u67051
https://www.next.co.uk/g224209s6/d99802#d99802
https://www.next.co.uk/style/st548096/368627#368627
https://www.next.co.uk/style/st548096/d35916#d35916
https

In [ ]:
driver.quit()

In [ ]:

# for data in final_all_scrapped:
#   print(data)
# print(len(final_all_scrapped))